In [72]:
using JuMP, GLPK
using DataFrames
using CSV

In [73]:
##Creation of the Input DataFrames

#load demand data
Demands = CSV.File("../data Assign 1/Demands.csv") |> DataFrame

#load conventional generators data
Conv_gen = CSV.File("../data Assign 1/Conventional_generators.csv") |> DataFrame

#load wind farms data
Wind_farms = CSV.File("../data Assign 1/Wind_farms.csv") |> DataFrame

#load transmission lines data
Trans_lines = CSV.File("../data Assign 1/transmission_lines.csv") |> DataFrame

Row,Transmission lines: From node,To node,Susceptance [per-unit],Capacity [MW]
,Int64,Int64,String7,Int64
1,1,2,0.0146,175
2,1,3,0.2253,175
3,1,5,0.0907,350
4,2,4,0.1356,175
5,2,6,0.205,175
6,3,9,0.1271,175
7,3,24,0.084,400
8,4,9,0.111,175
9,5,10,0.094,350


In [74]:
##Definition of the Result DataFrames

# Definition of the Main result DataFrame
main_result = DataFrame(hour = Int[], social_welfare = Float64[], market_price = Float64[])

# Initialize the Demand DataFrame directly without dynamic column names
details_results_demand = DataFrame(hour = Int[], total_demand = Float64[])
# For x variables, manually add each column. This is a one-time setup.
for i in 1:size(Demands, 1)
    details_results_demand[!, Symbol("demand_n°$i")] = Float64[]
end

# Initialize the Supply DataFrame directly without dynamic column names
details_results_supply = DataFrame(hour = Int[], total_supply = Float64[])
# For x variables, manually add each column. This is a one-time setup.
for i in 1:size(Conv_gen, 1)
    details_results_supply[!, Symbol("supply_n°$i")] = Float64[]
end
for i in size(Conv_gen, 1):(size(Conv_gen, 1)+size(Wind_farms, 1))
    details_results_supply[!, Symbol("supply_n°$i")] = Float64[]
end


In [75]:
##Definition of the model

# Create a new model with GLPK solver
model = Model(GLPK.Optimizer)


A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK

In [76]:
##Computation of the optimization problem for every hour

#for hour in 1:24
    # Define the decision variables for every supply and every demand
    unregister(model, :x)  # Unregister the existing variable named "x" from the model
    @variable(model, x[1:size(Conv_gen, 1) + size(Wind_farms, 1) + size(Demands, 1)])  # Create the new variables

    for i in 1:size(Conv_gen, 1)
        set_lower_bound(x[i], 0)  # Set the lower bound
        set_upper_bound(x[i], Conv_gen[i,6])  # Set the upper bound
    end
    for i in (size(Conv_gen, 1) + 1):(size(Conv_gen, 1)+size(Wind_farms, 1))
        set_lower_bound(x[i], 0)  # Set the lower bound
        set_upper_bound(x[i], Wind_farms[i - size(Conv_gen, 1),4])  # Set the upper bound
    end
    for i in (size(Conv_gen, 1)+size(Wind_farms, 1) + 1):(size(Conv_gen, 1) + size(Wind_farms, 1) + size(Demands, 1))
        set_lower_bound(x[i], 0)  # Set the lower bound
        set_upper_bound(x[i], Demands[i - size(Conv_gen, 1) - size(Wind_farms, 1),3])  # Set the upper bound
    end

    # Define the objective function
    @objective(model, Min, 
    - (sum(Demands[demand,4] * x[demand + size(Conv_gen, 1) + size(Wind_farms, 1)] for demand in 1:size(Demands,1))
    - (sum(Conv_gen[generator,3] * x[generator] for generator in 1:size(Conv_gen,1))
    + sum( 0 * x[generator + size(Conv_gen,1)] for generator in 1:size(Wind_farms,1)))))

     # Add the equilibrium constraint
     @constraint(model, sum(x[generator] for generator in 1:size(Conv_gen,1)+size(Wind_farms,1)) == sum(x[demand + size(Conv_gen, 1) + size(Wind_farms, 1)] for demand in 1:size(Demands,1)))

    # Print the model
    #print(model)

    # Solve the model
    optimize!(model)


    

In [ ]:
TBC

In [ ]:
    # Define the objective function
    @objective(model, Min, sum(cost[generator,2] * x[generator] for generator in 1:size(cost,1)))

    # Add the equilibrium constraint
    @constraint(model, sum(x[generator] for generator in 1:size(cost,1)) == demand[hour,2])

    # Print the model
    #print(model)

    # Solve the model
    optimize!(model)

    # Check the status of the solution
    status = termination_status(model)
    if status == MOI.OPTIMAL
        println("Hour ", hour, ": Optimal solution found")

        # Assuming 'model' and 'x' are properly defined and 'hour' is known
        new_row = [hour, objective_value(model)]
        append!(new_row, [value(x[i]) for i in 1:size(cost, 1)]) # Simplifies adding x values

        push!(result_df, new_row)

    elseif status == MOI.INFEASIBLE
        println("Hour ", hour, ": No feasible solution found")
    else
        println("Hour ", hour, ": Solver status: ", status)
    end
end



In [21]:
result_df

Row,hour,objective_value,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,15192.9,30.4,30.4,75.0,206.85,12.0,54.25,54.25,364.185,400.0,300.0,108.5,140.0
2,2,14554.6,30.4,30.4,75.0,206.85,12.0,54.25,54.25,258.165,400.0,300.0,108.5,140.0
3,3,14076.0,30.4,30.4,75.0,206.85,12.0,54.25,54.25,178.65,400.0,300.0,108.5,140.0
4,4,13916.4,30.4,30.4,75.0,206.85,12.0,54.25,54.25,152.145,400.0,300.0,108.5,140.0
5,5,13916.4,30.4,30.4,75.0,206.85,12.0,54.25,54.25,152.145,400.0,300.0,108.5,140.0
6,6,14076.0,30.4,30.4,75.0,206.85,12.0,54.25,54.25,178.65,400.0,300.0,108.5,140.0
7,7,16983.5,30.4,30.4,75.0,206.85,12.0,155.0,103.22,400.0,400.0,300.0,108.5,140.0
8,8,20353.5,30.4,30.4,75.0,206.85,12.0,155.0,155.0,400.0,400.0,300.0,310.0,204.78
9,9,23178.0,30.4,123.725,75.0,206.85,12.0,155.0,155.0,400.0,400.0,300.0,310.0,350.0


In [22]:
#save result_df to csv
CSV.write("../results/market_clearing_Jan.csv", result_df)

"../results/market_clearing_Jan.csv"